In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls
%cd gdrive/My Drive/final_project/final-project/collab

In [ ]:
!pip install transformers

In [ ]:
import torch
# specify GPU
device = torch.device("cuda")
torch.cuda.get_device_name(0)

In [ ]:
import preprocess as pproc
import dataprocess as dproc

In [ ]:
df = pproc.load_data("./dataset.csv")

In [ ]:
MAX_LEN = 512
# define a batch size
batch_size = 32
# learning rate
learning_rate = 2e-5
epochs = 4

In [ ]:
model, optimizer, class_wts, train_dataloader, val_dataloader, test_dataloader = dproc.data_processing(df, MAX_LEN, learning_rate, batch_size)

Load model        ...



Elapsed time: 11.146
Split data        ... Elapsed time: 0.059
Tokenize          ... Elapsed time: 50.586
Seq to Tensor     ... Elapsed time: 4.569
Create DataLoader ... Elapsed time: 0.000


In [ ]:
import torch.nn as nn

# push the model to GPU
model = model.to(device)

# convert class weight to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights)

In [ ]:
from timeit import default_timer as timer

best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
    
  print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
  start = timer()
  #train model
  train_loss, _ = dproc.train(device, model, optimizer, cross_entropy, train_dataloader)
  
  #evaluate model
  valid_loss, _ = dproc.evaluate(device, model, cross_entropy, val_dataloader)
  
  #save the best model
  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'saved_weights.pt')
  
  # append training and validation loss
  train_losses.append(train_loss)
  valid_losses.append(valid_loss)
  
  print(f'\nTraining Loss: {train_loss:.3f}')
  print(f'Validation Loss: {valid_loss:.3f}')
  print("Epoch time: {:.5f}".format(timer()-start))